In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

import tensorflow as tf
from tensorflow.keras import layers, losses, Input
from tensorflow.keras.models import Model

2022-05-12 14:51:21.377311: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


Links:

- https://blog.keras.io/building-autoencoders-in-keras.html
- https://www.tensorflow.org/tutorials/generative/autoencoder


In [2]:
pretrain_features = pd.read_csv('handout/pretrain_features.csv')
pretrain_labels = pd.read_csv('handout/pretrain_labels.csv')
train_features = pd.read_csv('handout/train_features.csv')
train_labels = pd.read_csv('handout/train_labels.csv')
test_features = pd.read_csv('handout/test_features.csv')

## Cleaning data

In [3]:
for c in pretrain_features.columns[2:]:
    if len(pretrain_features[c].unique()) < 2:
        pretrain_features = pretrain_features.drop(c, axis=1)
        train_features = train_features.drop(c, axis=1)
        test_features = test_features.drop(c, axis=1)

## Autoencoder

In [4]:
class Autoencoder(Model):
    def __init__(self, input_dim, latent_dim):
        super(Autoencoder, self).__init__()
        self.latent_dim = latent_dim   
        self.encoder = tf.keras.Sequential([
            Input(shape=(input_dim,)),
            layers.Dense(256, activation='relu'),
            layers.Dense(128, activation='relu'),
            layers.Dense(latent_dim),
        ])
        self.decoder = tf.keras.Sequential([
            Input(shape=(latent_dim,)),
            layers.Dense(128, activation='relu'),
            layers.Dense(256, activation='relu'),
            layers.Dense(input_dim)
        ])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [5]:
X_pretrain_train, X_pretrain_val, y_pretrain_train, y_pretrain_val = train_test_split(
    pretrain_features[pretrain_features.columns[2:]], pretrain_labels[["lumo_energy"]], test_size=0.1, random_state=42
)


In [6]:
autoencoder = Autoencoder(X_pretrain_train.shape[1], latent_dim=32)
autoencoder.compile(optimizer='adam', loss=losses.MSE)

2022-05-12 14:51:44.839672: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-05-12 14:51:44.840755: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-12 14:51:44.840782: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-12 14:51:44.840818: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (t3ui01.psi.ch): /proc/driver/nvidia/version does not exist
2022-05-12 14:51:44.841291: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild Ten

In [7]:
history = autoencoder.fit(X_pretrain_train, X_pretrain_train, epochs=10, batch_size=32, validation_data=(X_pretrain_val, X_pretrain_val))

2022-05-12 14:51:46.715479: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 350640000 exceeds 10% of free system memory.
2022-05-12 14:51:47.227424: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 350640000 exceeds 10% of free system memory.
2022-05-12 14:51:47.497465: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-05-12 14:51:47.498060: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2294660000 Hz


Epoch 1/10
1407/1407 [==============================] - 5s 3ms/step - loss: 0.0227 - val_loss: 0.0142
Epoch 2/10
1407/1407 [==============================] - 4s 2ms/step - loss: 0.0137 - val_loss: 0.0124
Epoch 3/10
1407/1407 [==============================] - 4s 3ms/step - loss: 0.0121 - val_loss: 0.0114
Epoch 4/10
1407/1407 [==============================] - 4s 3ms/step - loss: 0.0111 - val_loss: 0.0107
Epoch 5/10
1407/1407 [==============================] - 3s 2ms/step - loss: 0.0104 - val_loss: 0.0102
Epoch 6/10
1407/1407 [==============================] - 4s 3ms/step - loss: 0.0100 - val_loss: 0.0098
Epoch 7/10
1407/1407 [==============================] - 4s 3ms/step - loss: 0.0096 - val_loss: 0.0095
Epoch 8/10
1407/1407 [==============================] - 4s 3ms/step - loss: 0.0094 - val_loss: 0.0093
Epoch 9/10
1407/1407 [==============================] - 4s 3ms/step - loss: 0.0092 - val_loss: 0.0091
Epoch 10/10
1407/1407 [==============================] - 4s 3ms/step - loss: 0.009

## PreModel

In [8]:
class ELumon(Model):
    def __init__(self, autoencoder):
        super(ELumon, self).__init__()
        self.encoder = autoencoder.encoder
        self.regressor = tf.keras.Sequential([
            Input(shape=self.encoder.output_shape[1:]),
            layers.BatchNormalization(),
            layers.Dense(1, activation='linear', kernel_regularizer="l2")
        ])
        # Set all layers to trainable
        for layer in self.encoder.layers:
            layer.trainable = True

    def call(self, x):
        encoded = self.encoder(x)
        return self.regressor(encoded)

In [9]:
elumon = ELumon(autoencoder)
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
elumon.compile(optimizer=opt, loss="mean_squared_error")

In [10]:
elumon.fit(X_pretrain_train, y_pretrain_train, epochs=10, batch_size=32, validation_data=(X_pretrain_val, y_pretrain_val))

2022-05-12 14:52:38.118162: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 350640000 exceeds 10% of free system memory.


Epoch 1/10
1407/1407 [==============================] - 3s 2ms/step - loss: 9.3644 - val_loss: 3.2903
Epoch 2/10
1407/1407 [==============================] - 3s 2ms/step - loss: 1.6524 - val_loss: 0.1166
Epoch 3/10
1407/1407 [==============================] - 3s 2ms/step - loss: 0.1276 - val_loss: 0.0821
Epoch 4/10
1407/1407 [==============================] - 3s 2ms/step - loss: 0.0937 - val_loss: 0.0761
Epoch 5/10
1407/1407 [==============================] - 3s 2ms/step - loss: 0.0800 - val_loss: 0.0675
Epoch 6/10
1407/1407 [==============================] - 3s 2ms/step - loss: 0.0734 - val_loss: 0.0638
Epoch 7/10
1407/1407 [==============================] - 3s 2ms/step - loss: 0.0684 - val_loss: 0.0592
Epoch 8/10
1407/1407 [==============================] - 3s 2ms/step - loss: 0.0636 - val_loss: 0.0557
Epoch 9/10
1407/1407 [==============================] - 3s 2ms/step - loss: 0.0585 - val_loss: 0.0530
Epoch 10/10
1407/1407 [==============================] - 3s 2ms/step - loss: 0.055

## Model

In [11]:
class EGap(Model):
    def __init__(self, premodel):
        super(EGap, self).__init__()
        self.encoder = premodel.encoder
        self.regressor = tf.keras.Sequential([
            Input(shape=self.encoder.output_shape[1:]),
            layers.BatchNormalization(),
            layers.Dense(1, activation='linear', kernel_regularizer="l2")
        ])
        # Set all layers to trainable
        for layer in self.encoder.layers:
            layer.trainable = True

    def call(self, x):
        encoded = self.encoder(x)
        return self.regressor(encoded)

In [12]:
egap = EGap(elumon)
egap.compile(optimizer=opt, loss="mean_squared_error")

In [14]:
X_train_train, X_train_val, y_train_train, y_train_val = train_test_split(
    train_features[train_features.columns[2:]], train_labels[["homo_lumo_gap"]], test_size=0.02, random_state=42
)

In [17]:
egap.fit(X_train_train, y_train_train, epochs=20, batch_size=32, validation_data=(X_train_val, y_train_val))

Epoch 1/20
4/4 [==============================] - 1s 54ms/step - loss: 4.7848 - val_loss: 1.8250
Epoch 2/20
4/4 [==============================] - 0s 11ms/step - loss: 4.3233 - val_loss: 3.9508
Epoch 3/20
4/4 [==============================] - 0s 11ms/step - loss: 3.8178 - val_loss: 6.4901
Epoch 4/20
4/4 [==============================] - 0s 11ms/step - loss: 3.4682 - val_loss: 8.7711
Epoch 5/20
4/4 [==============================] - 0s 11ms/step - loss: 3.2948 - val_loss: 11.0489
Epoch 6/20
4/4 [==============================] - 0s 13ms/step - loss: 3.1725 - val_loss: 12.8866
Epoch 7/20
4/4 [==============================] - 0s 18ms/step - loss: 3.0200 - val_loss: 13.8013
Epoch 8/20
4/4 [==============================] - 0s 15ms/step - loss: 3.1236 - val_loss: 13.4427
Epoch 9/20
4/4 [==============================] - 0s 18ms/step - loss: 2.9095 - val_loss: 12.8805
Epoch 10/20
4/4 [==============================] - 0s 14ms/step - loss: 2.7737 - val_loss: 12.5843
Epoch 11/20
4/4 [======

In [19]:
y_pred = egap.predict(test_features[test_features.columns[2:]])

In [27]:
output_df = pd.DataFrame({"Id": test_features["Id"], "y": y_pred.reshape(-1,)})
output_df.to_csv("submission.csv", index=False)